# Multivariate Temporal Dictionary Learning for EEG
In this notebook, we reimplement experiment 1 of the article : 
Q. Barthélemy, C. Gouy-Pailler, Y. Isaac, A. Souloumiac, A. Larue, J.I. Mars,
Multivariate temporal dictionary learning for EEG,
Journal of Neuroscience Methods,
Volume 215, Issue 1,
2013,
Pages 19-28,
ISSN 0165-0270,
https://doi.org/10.1016/j.jneumeth.2013.02.001.
(https://www.sciencedirect.com/science/article/pii/S0165027013000666)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# files handling
import pickle
from os import listdir
#from os.path import exists
#from scipy.io import loadmat
import mne

# Dataset

The dataset used in this experiment is dataset 2.a of the 2008 BCI competition. A description of this dataset can be found here : http://bbci.de/competition/iv/desc_2a.pdf . Although this dataset was made for a classification task, we will use it for dictionnary learning on EEG signals.

### Load dataset
For the preprocessing, we follow the article, and apply a bandwidth filter [8hz, 30Hz]. We also discard 3 channels : 'EOG-left', 'EOG-right', and 'EOG-central', as these channels are not useful to our task.

In [29]:
def load_dataset(preprocessing=True):
    train = []
    test = []
    path = './dataset/'
    items = listdir(path)
    for item in items :
        #Load session
        raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)
        # print(raw.info.ch_names)
        
        # Apply bandwith filter
        if preprocessing:
            raw_filtered = raw.copy().filter(l_freq=8, h_freq=30, picks=None)
        # Convert to ndarray and discard EOG channels
        raw_eeg = raw_filtered.get_data(picks=
                                       ['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 
                                        'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9',
                                        'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16'])
        #raw_eeg.plot(n_channels=22)
        # Store in corresponding list
        if item[-5:] == 'T.gdf':
            train.append(raw_eeg)
        if item[-5:] == 'E.gdf':
            test.append(raw_eeg)
    return(train, test)

train, test = load_dataset(preprocessing=True)

C:\Users\viniv\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-29-6e58d04f6383>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)


['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


C:\Users\viniv\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-29-6e58d04f6383>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)


['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


C:\Users\viniv\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-29-6e58d04f6383>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)


...

['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


C:\Users\viniv\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-29-6e58d04f6383>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)


['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


C:\Users\viniv\Anaconda3\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-29-6e58d04f6383>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf(path+item, verbose=False, preload=True)


['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


In [30]:
print(train[0].shape)

(22, 672528)


# Multivariate dictionary learning